## Models available in the API and how to use them

The API can serve one or more trained models. By default this project ships a Random Forest model identified as `rf` (saved under `api/models/wine_classifier.pkl`).

To see which models are currently loaded, call the `/models` endpoint (GET) which returns the active model name(s) and required feature names.

How to select a model in requests:

- For single predictions (`POST /predict`) include the optional `model` key in the JSON body, e.g. `{ "model": "rf", "features": { ... } }`.
- For batch predictions (`POST /predict/batch`) include `model` in the batch body, e.g. `{ "model": "rf", "samples": [ {...}, {...} ] }`.
- If you omit `model`, the API will use the default model it has loaded.

If you need to use a different model file, place it in `api/models` and restart the server so the API can load it.

## Start the FastAPI server

Run this from the project root (activate your virtual environment first if needed):

.\.venv\Scripts\activate
>> python wine_predict/pipeline.py 

```powershell
uvicorn api.api_fastapi:app --host 127.0.0.1 --port 8000 --reload
```

After starting, open the interactive docs at: http://127.0.0.1:8000/docs

## Single-sample prediction — Example requests

Use the `/predict` endpoint to send a single datapoint. The API expects a JSON object with two keys: `model` (optional) and `features`. `features` should include the same numeric fields as the training data:

`fixed_acidity`, `volatile_acidity`, `citric_acid`, `residual_sugar`, `chlorides`, `free_sulfur_dioxide`, `total_sulfur_dioxide`, `density`, `pH`, `sulphates`, `alcohol`.

**PowerShell (Invoke-RestMethod)** — single prediction example:

```powershell
# Build JSON payload and POST in one line (uses -Depth to serialize nested objects)
Invoke-RestMethod -Uri 'http://127.0.0.1:8000/predict' -Method POST -Body (ConvertTo-Json @{ model = 'rf'; features = @{ fixed_acidity = 7.4; volatile_acidity = 0.7; citric_acid = 0.0; residual_sugar = 1.9; chlorides = 0.076; free_sulfur_dioxide = 11.0; total_sulfur_dioxide = 34.0; density = 0.9978; pH = 3.51; sulphates = 0.56; alcohol = 9.4 } } -Depth 5) -ContentType 'application/json'
```

In [ ]:
# Single-sample prediction (Python `requests`)
import requests, json
payload = {
    "model": "rf",
    "features": {
        "fixed_acidity": 7.4,
        "volatile_acidity": 0.7,
        "citric_acid": 0.0,
        "residual_sugar": 1.9,
        "chlorides": 0.076,
        "free_sulfur_dioxide": 11.0,
        "total_sulfur_dioxide": 34.0,
        "density": 0.9978,
        "pH": 3.51,
        "sulphates": 0.56,
        "alcohol": 9.4
    }
}
resp = requests.post('http://127.0.0.1:8000/predict', json=payload)
print('status:', resp.status_code)
try:
    print(json.dumps(resp.json(), indent=2))
except ValueError:
    print(resp.text)

status: 200
{
  "model_used": "RandomForestClassifier",
  "prediction": {
    "class": "average",
    "probability": {
      "average": 0.9261138050631545,
      "good": 0.017523722138444678,
      "poor": 0.05636247279840084
    },
    "confidence": 0.9261138050631545
  },
  "input_features": {
    "fixed_acidity": 7.4,
    "volatile_acidity": 0.7,
    "citric_acid": 0.0,
    "residual_sugar": 1.9,
    "chlorides": 0.076,
    "free_sulfur_dioxide": 11.0,
    "total_sulfur_dioxide": 34.0,
    "density": 0.9978,
    "pH": 3.51,
    "sulphates": 0.56,
    "alcohol": 9.4
  }
}


## /predict (multiple samples)

Use `POST /predict/batch` to send a list of samples in the `samples` field. The following Python cell builds a valid JSON payload you can inspect or save to file.

In [ ]:
# Editable JSON generator for `/predict/batch`.
# You can include more than two samples — append additional sample dicts to the list.
import json
batch = {
    "model": "rf",
    "samples": [
        {
            "fixed_acidity": 7.4,
            "volatile_acidity": 0.7,
            "citric_acid": 0.0,
            "residual_sugar": 1.9,
            "chlorides": 0.076,
            "free_sulfur_dioxide": 11.0,
            "total_sulfur_dioxide": 34.0,
            "density": 0.9978,
            "pH": 3.51,
            "sulphates": 0.56,
            "alcohol": 9.4
        },
        {
            "fixed_acidity": 6.0,
            "volatile_acidity": 0.6,
            "citric_acid": 0.12,
            "residual_sugar": 2.5,
            "chlorides": 0.05,
            "free_sulfur_dioxide": 10.0,
            "total_sulfur_dioxide": 30.0,
            "density": 0.995,
            "pH": 3.3,
            "sulphates": 0.5,
            "alcohol": 10.1
        },
        {
            "fixed_acidity": 5.8,
            "volatile_acidity": 0.65,
            "citric_acid": 0.08,
            "residual_sugar": 2.2,
            "chlorides": 0.06,
            "free_sulfur_dioxide": 9.0,
            "total_sulfur_dioxide": 28.0,
            "density": 0.9965,
            "pH": 3.45,
            "sulphates": 0.52,
            "alcohol": 9.8
        }
    ]
}
print(json.dumps(batch, indent=2))

{
  "model": "rf",
  "samples": [
    {
      "fixed_acidity": 7.4,
      "volatile_acidity": 0.7,
      "citric_acid": 0.0,
      "residual_sugar": 1.9,
      "chlorides": 0.076,
      "free_sulfur_dioxide": 11.0,
      "total_sulfur_dioxide": 34.0,
      "density": 0.9978,
      "pH": 3.51,
      "sulphates": 0.56,
      "alcohol": 9.4
    },
    {
      "fixed_acidity": 6.0,
      "volatile_acidity": 0.6,
      "citric_acid": 0.12,
      "residual_sugar": 2.5,
      "chlorides": 0.05,
      "free_sulfur_dioxide": 10.0,
      "total_sulfur_dioxide": 30.0,
      "density": 0.995,
      "pH": 3.3,
      "sulphates": 0.5,
      "alcohol": 10.1
    },
    {
      "fixed_acidity": 5.8,
      "volatile_acidity": 0.65,
      "citric_acid": 0.08,
      "residual_sugar": 2.2,
      "chlorides": 0.06,
      "free_sulfur_dioxide": 9.0,
      "total_sulfur_dioxide": 28.0,
      "density": 0.9965,
      "pH": 3.45,
      "sulphates": 0.52,
      "alcohol": 9.8
    }
  ]
}
